In [634]:
# !IPython.load_extensions('calico-document-tools') To construct notebook Table of Contents
import numpy as np
import pandas as pd
import pysal as ps
import scipy
from scipy.integrate import quad
from scipy.stats import norm
from collections import Counter

from scipy.stats import beta # Beta probability function
from scipy.special import digamma
from scipy.special import beta as f_beta # Beta raw function

import os

os.chdir('C:/Users/renan/Desktop/inequality-segregation-supplementary-files/')
census_2010 = pd.read_csv('data/std_2010_fullcount.csv', encoding = "ISO-8859-1", sep = ",")
census_2010.head()

,trtid10,state,county,tract,pop10,nhwht10,nhblk10,ntv10,asian10,hisp10,...,a15hsp10,a60hsp10,ageasn10,a15asn10,a60asn10,agentv10,a15ntv10,a60ntv10,globd10,globg10
0,1001020100,AL,Autauga County,Census Tract 201,1912,1601,228,21,16,44,...,14,2,14,4,1,13,1,3,bw,White Black
1,6083002402,CA,Santa Barbara County,Census Tract 24.02,11406,1980,207,54,703,8439,...,2972,414,624,119,75,26,4,3,wha,Dual immig
2,1001020200,AL,Autauga County,Census Tract 202,2170,844,1226,9,13,75,...,14,1,5,0,0,5,3,1,bw,White Black
3,6083002102,CA,Santa Barbara County,Census Tract 21.02,2084,853,24,24,88,1088,...,342,84,61,4,16,11,0,4,wha,Dual immig
4,1001020300,AL,Autauga County,Census Tract 203,3373,2538,668,30,42,87,...,34,9,22,2,7,9,1,3,bw,White Black


In [2]:
def duncan_betamix(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]

    if (Cmix == 1):
        lam = 1 
    else: 
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))

    value = sum(lam * (b * beta.cdf(mu, a, b + 1)/(1 - mu) - a * beta.cdf(mu, a + 1, b) / mu) / (a + b))
    return value

In [3]:
duncan_betamix([0.3, 0.7])

0.6657228773303705

In [4]:
duncan_betamix([0.3, 0.7, 0.04, 0.65, 0.7])

0.8075080072970819

In [5]:
duncan_betamix([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.7403564967329468

In [138]:
def theil_betamix_2017(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]
    
    if (Cmix == 1):
        lam = 1 
    else:
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))

    value = 1 - sum(lam * (a / (a + b) * digamma(a + 1) - a / (a + b) * digamma(a + b + 1))) / (mu * np.log(mu))
    return value

In [139]:
theil_betamix_2017([0.3, 0.7])

0.5083151198778997

In [140]:
theil_betamix_2017([0.3, 0.7, 0.04, 0.65, 0.7])

0.5348452644472066

In [141]:
theil_betamix_2017([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.6219157169201162

In [149]:
def theil_betamix_2012(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]
    
    if (Cmix == 1):
        lam = 1 
    else:
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))

    value = 1 - sum(lam * (a / (a + b) * digamma(a + 1) + b / (a + b) * digamma(b + 1) - digamma(a + b + 1))) / (mu * np.log(mu) + (1 - mu) * np.log(1 - mu))
    return value

In [150]:
theil_betamix_2012([0.3, 0.7])

0.46377929067849466

In [151]:
theil_betamix_2012([0.3, 0.7, 0.04, 0.65, 0.7])

0.6436871528489695

In [152]:
theil_betamix_2012([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.5568732745587515

In [10]:
def gini_betamix(v):
    v = np.array(v)              # Has to be np.array for future calculations
    Cmix = int((1 + len(v)) / 3) # Has to be integer for future slices
    a = v[0:Cmix]      
    b = v[(Cmix):(2 * Cmix)]
    
    if (Cmix == 1):
        lam = np.array([1]) 
    else:
        lam = np.append(v[(2 * Cmix):(3 * Cmix - 1)], 1 - sum(v[(2 * Cmix):(3 * Cmix - 1)]))
        
    mu = sum(lam * a / (a + b))
    

    def ff(x):
        res = np.empty(shape = (Cmix, 2))
        for ii in range(Cmix):
            res[ii,0] = lam[ii] * b[ii] * beta.pdf(x, a[ii], b[ii] + 1) / (a[ii] + b[ii])
            res[ii,1] = lam[ii] * a[ii] * beta.cdf(x, a[ii] + 1, b[ii]) / (a[ii] + b[ii])
        res = np.sum(res, axis = 0) # apply(res, c(2, 3), sum)
        
        return (res[0] * res[1])

    value = 1-(2/(mu*(1-mu)))*(quad(ff,0,.5,limit=100)[0]+quad(ff,.5,1,limit=100)[0])
    return value


In [11]:
gini_betamix([0.3, 0.7])

0.834506614191632

In [12]:
gini_betamix([0.3, 0.7, 0.04, 0.65, 0.7])

0.9342811402444533

In [13]:
gini_betamix([0.3, 0.7, 0.04, 0.65, 0.7, 0.9, 0.12, 0.43])

0.891209223701447

In [216]:
def fmn(a,b,m,n):
    value = f_beta(a + n, b + m - n) / f_beta(a, b)
    return value

In [273]:
fmn(0.3, 0.4, 7, 3)

0.0019209261240983927

In [277]:
def llik_cnt(a, b, lam, K, X):
    # a = vector of size C (number of components of the mixture)
    # b = vector of size C (number of components of the mixture)
    # lam = vector of size C (number of components of the mixture)
    # K = scalar; number of indiv by unit
    # X = vector of size K+1; number of units with (j-1) minorities
    
    L = np.array(np.repeat(lam, K+1)).reshape((len(lam), K+1)) # Repeat each element of the list K+1 times, since Python fills rowwisely
    A = np.array(np.repeat(a,   K+1)).reshape((len(lam), K+1))
    B = np.array(np.repeat(b,   K+1)).reshape((len(lam), K+1))
    
    Kmat = np.array(list(range(K + 1)) * len(lam)).reshape((len(lam), K + 1))

    value = -sum(X * np.log(np.sum(np.multiply(L, fmn(A, B, K, Kmat)), axis = 0))) # multiply element wisely
    return value

In [279]:
a = [0.3, 0.6]
b = [0.44, 0.9]
lam = [0.45, 0.55]
K = 5
X = [6, 9, 10, 4, 8, 7]
llik_cnt(a, b, lam, K, X)

142.61710934389515

In [338]:
def llik_Kvec(a, b, lam, Xmat, Kvec):
    cumsum = 0
    
    for j in list(range(len(Kvec))):
        cumsum += llik_cnt(a, b, lam, Kvec[j], Xmat[0:(Kvec[j]+1), j])
    
    return cumsum


In [339]:
Kvec = [3, 2]

values = np.linspace(1, 20, num = 20)
Xmat = np.array(values).reshape((int(len(values) / len(Kvec)), len(Kvec)))
Xmat

array([[ 1.,  2.],
       [ 3.,  4.],
       [ 5.,  6.],
       [ 7.,  8.],
       [ 9., 10.],
       [11., 12.],
       [13., 14.],
       [15., 16.],
       [17., 18.],
       [19., 20.]])

In [342]:
llik_Kvec(a,b,lam,Xmat,Kvec)

51.36943404147456

In [463]:
def moments_beta(a, b, lam, K):
    C = len(a)
    A = np.matmul(np.array([1] * (K + 1)).reshape(K+1,1), np.array(a).reshape(1,len(a)))
    B = np.matmul(np.array([1] * (K + 1)).reshape(K+1,1), np.array(b).reshape(1,len(b)))
    # np.repeat: repeat each element
    # np.tile: repeat entire np.array
    mm = f_beta(np.add(A, np.repeat(np.linspace(0, K, K + 1), C).reshape(K+1, C)), B) / f_beta(A, B)
    L = np.matmul(np.array([1] * (K + 1)).reshape(K+1,1), np.array(lam).reshape(1, len(lam)))
    value = np.sum(np.multiply(mm, L), axis = 1)[1:]
    return value

In [464]:
moments_beta(a, b, lam, K)

array([0.40243243, 0.27710009, 0.21900677, 0.18462762, 0.16156419])

In [490]:
np.random.normal(size = 5)

8103.083927575384

In [524]:
norm.cdf(0.9)

0.8159398746532405

In [ ]:
def initial_values(rawmom, Cmix):
    if (Cmix==1):
        m1  = rawmom[0]
        q2  = rawmom[1]/rawmom[0]
        res = np.linalg.solve(np.column_stack((np.array([m1 - 1, q2 - 1]),np.array([m1, q2]))),np.array([0,1-q2]))
        if (np.sum(res > 0) == 2):
            return res
        if (np.sum(res > 0) == 0):
            return -res
        if (np.sum(res > 0) == 1):
            return m1/(1 - m1)
  
  
    if (Cmix>=2):
        K = 3 * Cmix - 1
        empmom = rawmom[0:K]
        init = [np.log([1, 
                        0.2, 
                        np.exp(np.random.normal(size = Cmix-1) + 0), # Changed the original code here which was: Cmix - 2
                        1,
                        5,
                        np.exp(np.random.normal(size = Cmix-1))]), # Changed the original code here which was: Cmix - 2
                [norm.ppf(1/Cmix)] * (Cmix-1)]

        #Reparameterized
        def fnmin(x):
            value = np.sum((moments_beta(np.exp(x[0:Cmix]),
                                 np.exp(x[(Cmix):(2*Cmix)]),
                                 np.array(pnorm(x[(2*Cmix):K]),1-np.sum(norm.cdf(x[(2*Cmix):K]))),
                                 K)
                        -empmom)**2)
            return value

        def grmin(x):
            a   = np.exp(x[0:Cmix]) #x[1:Cmix]
            b   = np.exp(x[(Cmix):(2*Cmix)]) #x[(Cmix+1):(2*Cmix)]
            lam = np.array(norm.cdf(x[(2*Cmix):(3*Cmix-1)]),1-sum(norm.cdf(x[(2*Cmix):(3*Cmix-1)])))
            #c(x[(2*Cmix+1):(3*Cmix-1)],1-sum(x[(2*Cmix+1):(3*Cmix-1)]))
            A = np.tile(a, K).reshape(K, len(a))
            B = np.tile(b, K).reshape(K, len(b))
            Kmat = np.repeat(range(1, K+1), Cmix).reshape(K, Cmix)
            mm = f_beta(A+Kmat,B)/f_beta(A,B)
            diga = digamma(A+Kmat)-digamma(A+B+Kmat)+digamma(A+B)-digamma(A)
            digb = digamma(B)-digamma(A+B+Kmat)+digamma(A+B)-digamma(B)
            L = np.tile(lam, K).reshape((K, len(lam)))

            deralpha = A*L*mm*diga #L*mm*diga
            derbeta  = B*L*mm*digb #L*mm*digb
            derlam   = np.delete(norm.pdf(norm.ppf(L)), mm.shape[1] + 1, axis = 1)*(mm[,-ncol(mm)]-mm[,ncol(mm)]%x%t(rep(1,ncol(mm)-1)))
            return( as.numeric(2*(moments_beta(a,b,lam,K)-empmom)%*%cbind(deralpha,derbeta,derlam)) )
                             

        res=optim(init,fn=fnmin,gr=grmin,method="BFGS",
        control=list(maxit=1000,trace=0))

        return(c(exp(res$par[1:(2*Cmix)]),pnorm(res$par[(2*Cmix+1):(length(res$par))])))
  


In [587]:
mm.shape[1]

2

In [586]:
norm.pdf(norm.ppf(L))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [581]:
            a   = np.exp(x[0:Cmix]) #x[1:Cmix]
            b   = np.exp(x[(Cmix):(2*Cmix)]) #x[(Cmix+1):(2*Cmix)]
            lam = np.array(norm.cdf(x[(2*Cmix):(3*Cmix-1)]),1-sum(norm.cdf(x[(2*Cmix):(3*Cmix-1)])))
            #c(x[(2*Cmix+1):(3*Cmix-1)],1-sum(x[(2*Cmix+1):(3*Cmix-1)]))
            A = np.tile(a, K).reshape(K, len(a))
            B = np.tile(b, K).reshape(K, len(b))
            Kmat = np.repeat(range(1, K+1), Cmix).reshape(K, Cmix)
            mm = f_beta(A+Kmat,B)/f_beta(A,B)
            diga = digamma(A+Kmat)-digamma(A+B+Kmat)+digamma(A+B)-digamma(A)
            digb = digamma(B)-digamma(A+B+Kmat)+digamma(A+B)-digamma(B)
            L = np.tile(lam, K).reshape((K, len(lam)))

            deralpha = A*L*mm*diga #L*mm*diga
            derbeta  = B*L*mm*digb #L*mm*digb

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [583]:
derbeta

array([[-0.10499359,         nan],
       [-0.02604522,         nan],
       [-0.00504579,         nan],
       [-0.00090298,         nan],
       [-0.00015714,         nan]])

In [563]:
np.tile(lam, K).reshape((K, len(lam)))

array([[0.45, 0.55],
       [0.45, 0.55],
       [0.45, 0.55],
       [0.45, 0.55],
       [0.45, 0.55]])

In [557]:
digb

array([[-0.00593476, -0.00029547],
       [-0.01183451, -0.00059086],
       [-0.01769966, -0.00088616],
       [-0.02353061, -0.00118137],
       [-0.02932776, -0.0014765 ]])

In [554]:
A+Kmat

array([[ 21.08553692, 404.42879349],
       [ 22.08553692, 405.42879349],
       [ 23.08553692, 406.42879349],
       [ 24.08553692, 407.42879349],
       [ 25.08553692, 408.42879349]])

In [535]:
A = np.matmul(np.array([1] * K).reshape((K, 1)), np.array(a).reshape((len(a), 1)))

ValueError: shapes (5,1) and (2,1) not aligned: 1 (dim 1) != 2 (dim 0)

In [536]:
np.array([1] * K).reshape((K, 1))

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [537]:
np.array(a).reshape((len(a), 1))

array([[ 20.08553692],
       [403.42879349]])

In [526]:
        init = [np.log([1, 
                        0.2, 
                        np.exp(np.random.normal(size = Cmix-2) + 0), # Changed the original code here which was: Cmix - 2
                        1,
                        5,
                        np.exp(np.random.normal(size = Cmix-2))]), # Changed the original code here which was: Cmix - 2
                [norm.ppf(1/Cmix)] * (Cmix-1)]

ValueError: setting an array element with a sequence.

# Main Function

In [519]:
def betamix_estim(Xmat, Kvec, Cmax, initval = NULL):
  
    ## Xmat is the count matrix
    ## Make it a matrix, even if one column only
    if (type(Xmat) is not np.ndarray): 
        Xmat = np.array(Xmat).reshape((len(Xmat), 1))
    

    ## In the standard case, length(Kvec) = ncol(Xmat)
    ## This script also supports the case in which length(Kvec)<ncol(Xmat),
    ##   that is, when you want to restrict the estimation to a subset of the Ks
    ## number of rows >= max(Kvec)+2
    ## number of cols >= length(Kvec)
    ## each column has K+1 meaningful elements, other elements are negative
    ## for j=1...K+1, value(j) = nb of obs such that (X_K = j-1)

    ## Find K for each column
    whichK=apply(Xmat,2,function(v){min(which(v<0))})-2
    goodcol = which(whichK %in% Kvec)
    resm=matrix(0,nrow=max(Kvec)+1,ncol=length(goodcol))
    
    for (jj in 1:length(goodcol)):
        resm[1:sum(Xmat[,goodcol[jj]]>=0),jj]=Xmat[Xmat[,goodcol[jj]]>=0,goodcol[jj]]
    
    Xmat=resm
    
    if (length(Kvec)==1):
        Xmat=matrix(Xmat,ncol=1)
    

    ## Estimate moments from observed proportions 
    mm = synthesis.moments(Xmat,Kvec)

    reslist = list()

    ## 1 component
    # Compute good initial values for the likelihood
    Cmix=1
    if (is.null(initval[[1]])):
        init = initial.values(mm,Cmix)
        init = log(init)

    else:
        init = log(initval[[1]])
        
    # Max likelihood
    def aux_fun(x):
        return llik.Kvec(exp(x[1:Cmix]),exp(x[(Cmix+1):2*Cmix]),1,Xmat,Kvec)
    
    prem = optim(init,
                 fn = aux_fun,
                 method="BFGS",
                 hessian=FALSE)

    def fn2(x):
        llik.Kvec(x[1:Cmix],x[(Cmix+1):2*Cmix],1,Xmat,Kvec)

    reslist[[Cmix]]= optim(exp(prem$par),fn2,method="Nelder-Mead",hessian=FALSE)
    reslist[[Cmix]]$hessian = numDeriv::hessian(fn2,reslist[[Cmix]]$par)

    if (Cmax==1):
        return(reslist)

    ## 2 components and more
    ## For the moment, initial.values() goes up to Cmax=2
    for (Cmix in 2:Cmax):

        if (is.null(initval[[Cmix]])):
            init = initial.values(mm,Cmix)
        
        else:
            init = initval[[Cmix]]
        
        init = c(log(init[1:(2*Cmix)]),init[(2*Cmix+1):(3*Cmix-1)])
        
        def aux_fun2(x):
            return llik.Kvec(exp(x[1:Cmix]),exp(x[(Cmix+1):(2*Cmix)]),c(pnorm(x[(2*Cmix+1):(3*Cmix-1)]),1-sum(pnorm(x[(2*Cmix+1):(3*Cmix-1)]))),Xmat,Kvec)
        
        prem = optim(init,
                     fn = aux_fun2,
                     method="BFGS",
                     hessian=FALSE)

        init=c(exp(prem$par[1:(2*Cmix)]),pnorm(prem$par[(2*Cmix+1):(3*Cmix-1)]))

        def aux_fun3(x):
            return llik.Kvec(x[1:Cmix],x[(Cmix+1):(2*Cmix)],c(x[(2*Cmix+1):(3*Cmix-1)],1-sum(x[(2*Cmix+1):(3*Cmix-1)])),Xmat,Kvec)

        reslist[[Cmix]] = optim(init,
                                aux_fun3,
                                method="Nelder-Mead",
                                hessian=FALSE,
                                control=list(maxit=2))
        reslist[[Cmix]]$hessian = numDeriv::hessian(fn2,reslist[[Cmix]]$par)
        reslist[[Cmix]]$lr.pval = 1-pchisq(2*(reslist[[Cmix-1]]$value-reslist[[Cmix]]$value),3)


    return(reslist)


SyntaxError: invalid syntax (<ipython-input-519-958f99a1c2b9>, line 5)

In [591]:
os.chdir('C:\\Users\\renan\\Desktop\\RIDIR project\\Non_Parametric_Binomial_Rathelot_2017\\beta_mixture_approach_2012\\Archive')

In [593]:
data = pd.read_csv('simple.csv')
data.head()

,X,K
0,0,13
1,1,41
2,0,43
3,3,41
4,0,60


In [682]:
def vector_to_count(data, rangeK = None):
    
    # data is a pandas dataframe with columns X and K
    
    X = data['X']
    K = data['K']
    
    if (rangeK is None):
        rangeK = list(set(K.values))
    
    else:
        X = X[K.isin(rangeK)]
        K = K[K.isin(rangeK)]
    
    Pmat = np.full(shape = (max(K)+2, len(rangeK)), fill_value = -99)

    for j in range(len(rangeK)):
        Kj = rangeK[j]
        Xj = X[K == Kj]
        aux = Xj+1
        Pmat[0:(Kj+1),j] = np.bincount(aux, minlength=Kj+2)[1:] # tabulate(Xj+1,nbins=Kj+1)
    
    return {'Xmat': Pmat,
            'Kvec': rangeK}

In [681]:
aux = Xj+1
np.bincount(aux, minlength=Kj+2)[1:]

array([187,  20,   7,   2,   0], dtype=int64)

In [666]:
Pmat[0:(Kj+1),j]

array([-99, -99, -99, -99, -99])

In [667]:
l = [187, 20, 7, 2]

In [671]:
np.bincount(l)[1:]

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

In [664]:
j = 3
Kj = rangeK[j]
Xj = X[K == Kj]
list(Counter(Xj+1).values())#[0:(Kj+1)]

[187, 20, 7, 2]

In [665]:
list(Counter(Xj+1).values())

[187, 20, 7, 2]

In [683]:
vector_to_count(data, rangeK = None)

{'Xmat': array([[224, 296, 207, ...,   1,   0,   0],
        [ 15,  16,  17, ...,   0,   0,   1],
        [-99,   9,   7, ...,   0,   0,   0],
        ...,
        [-99, -99, -99, ..., -99, -99,   0],
        [-99, -99, -99, ..., -99, -99,   0],
        [-99, -99, -99, ..., -99, -99, -99]]),
 'Kvec': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  92,
  100,
  102,
  104,
  106,
  109,
  113,
  125,
  132,
  147]}

In [641]:
list(Counter(Xj+1).values())

[224, 15]

In [632]:
Xj = X[K == Kj]
Xj+1

82       1
85       1
103      1
122      1
163      1
167      1
195      1
252      1
300      1
353      1
373      1
406      1
427      1
431      1
448      1
488      2
505      2
512      2
541      1
574      1
597      1
679      1
692      1
876      1
904      1
956      1
964      1
973      1
981      1
1002     1
        ..
9706     1
9745     1
9759     1
9782     1
9829     1
9862     1
9922     1
9939     1
9944     1
10028    1
10062    1
10070    1
10090    1
10116    1
10168    1
10173    1
10181    1
10185    1
10298    1
10337    1
10370    1
10384    1
10438    1
10450    1
10458    1
10483    1
10490    1
10532    1
10541    1
10554    1
Name: X, Length: 239, dtype: int64

In [621]:
Pmat = np.full(shape = (max(K)+2, len(rangeK)), fill_value = -99)
Pmat.shape

(149, 98)

In [623]:
from collections import Counter
Counter([1,1,1,2,3])

Counter({1: 3, 2: 1, 3: 1})

In [624]:
?Counter

In [193]:
a = [0.3, 0.6]
b = [0.44, 0.9]
lam = [0.45, 0.55]
K = 5
X = [6, 9, 10, 4, 8, 7]

L = np.array(np.repeat(lam, K+1)) # Repeat each element of the list K+1 times, since Python fills rowwisely
L.shape = (len(lam), (K+1))
A = np.array(np.repeat(a, K+1)) # Repeat each element of the list K+1 times, since Python fills rowwisely
A.shape = (len(lam), (K+1))
B = np.array(np.repeat(a, K+1)) # Repeat each element of the list K+1 times, since Python fills rowwisely
B.shape = (len(lam), (K+1))
Kmat = np.array(list(range(K + 1)) * len(lam))
Kmat.shape = (len(lam), (K + 1))

In [194]:
Kmat

array([[0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5]])

In [195]:
fmn(A, B, K, Kmat)

array([[0.30794271, 0.02148438, 0.00846354, 0.00846354, 0.02148438,
        0.30794271],
       [0.22402597, 0.02922078, 0.01298701, 0.01298701, 0.02922078,
        0.22402597]])

In [115]:
K = 5
mat1K = np.matmul(np.matrix([[1]] * K), np.matrix(np.linspace(1, K, K)))
mat1K.T

matrix([[1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.]])

In [121]:
(-1)**(mat1K - mat1K.T)

TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'matrix'

In [111]:
np.matrix([[1], [1], [1], [1], [1]])

matrix([[1],
        [1],
        [1],
        [1],
        [1]])

In [92]:
(np.array([1] * K).T

SyntaxError: unexpected EOF while parsing (<ipython-input-92-5cc9727b2c4a>, line 1)

In [93]:
np.array([1] * K).T

array([1, 1])

In [90]:
np.transpose(np.linspace(1, K, K))

array([1., 2.])